In [28]:
import pandas as pd
import os
from pathlib import Path

# Đường dẫn gốc
BASE_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord")

# Định nghĩa các file cần gộp
FILES_CONFIG = {
    "Amazon": {
        "train": BASE_PATH / "Amazon" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Amazon" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Amazon" / "test_reviews_under80words.csv"
    },
    "Hotels": {
        "train": BASE_PATH / "Hotels" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Hotels" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Hotels" / "test_reviews_under80words.csv"
    },
    "AmazonsElectronics": {
        "all": BASE_PATH / "AmazonsElectronics" / "Kaggle" / "neutral_output_aspect.csv"
    }
}

print("Cấu hình đường dẫn files:")
for source, files in FILES_CONFIG.items():
    print(f"\n{source}:")
    for file_type, path in files.items():
        exists = "✓" if path.exists() else "✗"
        print(f"  {file_type}: {path.name} [{exists}]")

Cấu hình đường dẫn files:

Amazon:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

Hotels:
  train: train_reviews_under80words.csv [✓]
  dev: dev_reviews_under80words.csv [✓]
  test: test_reviews_under80words.csv [✓]

AmazonsElectronics:
  all: neutral_output_aspect.csv [✓]


In [29]:
def load_and_standardize_data(file_path):
    """
    Đọc file CSV và chuẩn hóa cột thành reviewText, AspectTerm, Sentiment
    
    Parameters:
    - file_path: đường dẫn file
    
    Returns:
    - DataFrame đã chuẩn hóa với 3 cột: reviewText, AspectTerm, Sentiment
    """
    try:
        df = pd.read_csv(file_path)
        
        # Chuẩn hóa tên cột thành chữ thường để dễ xử lý
        df.columns = df.columns.str.strip().str.lower()
        
        # Map các tên cột có thể khác nhau về tên chuẩn
        column_mapping = {
            'reviewtext': 'reviewText',
            'review': 'reviewText',
            'text': 'reviewText',
            'aspectterm': 'AspectTerm',
            'aspect': 'AspectTerm',
            'sentiment': 'Sentiment'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Chỉ giữ 3 cột cần thiết
        required_cols = ['reviewText', 'AspectTerm', 'Sentiment']
        available_cols = [col for col in required_cols if col in df.columns]
        df = df[available_cols]
        
        # Drop các dòng có AspectTerm trống hoặc null
        rows_before = len(df)
        if 'AspectTerm' in df.columns:
            df = df.dropna(subset=['AspectTerm'])
            df = df[df['AspectTerm'].str.strip() != '']
            df = df[df['AspectTerm'].str.lower() != 'none']
        rows_after = len(df)
        dropped = rows_before - rows_after
        
        print(f"  ✓ Loaded {rows_after} rows from {file_path.name} (dropped {dropped} rows with empty AspectTerm)")
        return df
        
    except Exception as e:
        print(f"  ✗ Error loading {file_path}: {e}")
        return pd.DataFrame()

print("Hàm load_and_standardize_data đã được định nghĩa!")

Hàm load_and_standardize_data đã được định nghĩa!


In [30]:
def merge_all_datasets(files_config, output_path=None, balance_config=None):
    """
    Gộp tất cả các dataset thành một file duy nhất
    
    Parameters:
    - files_config: dict chứa cấu hình các file
    - output_path: đường dẫn lưu file output (optional)
    - balance_config: dict để giới hạn số lượng mẫu theo sentiment (optional)
                      Ví dụ: {'positive': 5000, 'negative': 5000, 'neutral': 5000}
                      Nếu None, giữ nguyên tất cả dữ liệu
    
    Returns:
    - DataFrame đã gộp với 3 cột: reviewText, AspectTerm, Sentiment
    """
    all_dataframes = []
    
    print("=" * 60)
    print("BẮT ĐẦU GỘP DATASETS")
    print("=" * 60)
    
    for source_name, files in files_config.items():
        print(f"\n📂 Loading from {source_name}:")
        for file_type, file_path in files.items():
            if file_path.exists():
                df = load_and_standardize_data(file_path)
                if not df.empty:
                    all_dataframes.append(df)
            else:
                print(f"  ⚠ File not found: {file_path}")
    
    if not all_dataframes:
        print("\n❌ Không có dữ liệu nào được load!")
        return pd.DataFrame()
    
    # Gộp tất cả
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    
    # Áp dụng balance_config nếu có
    if balance_config:
        print("\n" + "=" * 60)
        print("ÁP DỤNG CÂN BẰNG DỮ LIỆU")
        print("=" * 60)
        
        balanced_dfs = []
        for sentiment, max_count in balance_config.items():
            sentiment_df = merged_df[merged_df['Sentiment'].str.lower() == sentiment.lower()]
            current_count = len(sentiment_df)
            
            if current_count > max_count:
                sentiment_df = sentiment_df.sample(n=max_count, random_state=42)
                print(f"  {sentiment}: {current_count:,} → {max_count:,} (giảm {current_count - max_count:,})")
            else:
                print(f"  {sentiment}: {current_count:,} (giữ nguyên, ít hơn {max_count:,})")
            
            balanced_dfs.append(sentiment_df)
        
        merged_df = pd.concat(balanced_dfs, ignore_index=True)
        merged_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ CUỐI CÙNG")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo Sentiment:")
    print(merged_df['Sentiment'].value_counts().to_string())
    print(f"\n📋 Các cột: {list(merged_df.columns)}")
    
    # Lưu file nếu có output_path
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"\n💾 Đã lưu file: {output_path}")
    
    return merged_df

print("Hàm merge_all_datasets đã được định nghĩa!")

Hàm merge_all_datasets đã được định nghĩa!


In [31]:
# ============================================
# CẤU HÌNH SỐ LƯỢNG MẪU THEO SENTIMENT
# ============================================
# Chỉnh sửa các giá trị bên dưới để thay đổi số lượng mẫu cho mỗi nhãn
# Đặt None nếu muốn giữ tất cả dữ liệu không giới hạn

BALANCE_CONFIG = {
    'positive': 3000,   # Số lượng mẫu positive tối đa
    'negative': 3000,   # Số lượng mẫu negative tối đa  
    'neutral': 1788    # Số lượng mẫu neutral tối đa
}

# Đặt BALANCE_CONFIG = None nếu muốn giữ tất cả dữ liệu
# BALANCE_CONFIG = None

# ============================================
# Thực hiện gộp dataset
OUTPUT_PATH = BASE_PATH / "merged_dataset_all_7700rows.csv"

merged_df = merge_all_datasets(
    FILES_CONFIG, 
    output_path=OUTPUT_PATH,
    balance_config=BALANCE_CONFIG
)

BẮT ĐẦU GỘP DATASETS

📂 Loading from Amazon:
  ✓ Loaded 6297 rows from train_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)
  ✓ Loaded 974 rows from dev_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)
  ✓ Loaded 966 rows from test_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)

📂 Loading from Hotels:
  ✓ Loaded 10126 rows from train_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)
  ✓ Loaded 1036 rows from dev_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)
  ✓ Loaded 242 rows from test_reviews_under80words.csv (dropped 0 rows with empty AspectTerm)

📂 Loading from AmazonsElectronics:
  ✓ Loaded 1212 rows from neutral_output_aspect.csv (dropped 3788 rows with empty AspectTerm)

KẾT QUẢ GỘP DATASET (TRƯỚC KHI CÂN BẰNG)

📊 Tổng số mẫu: 20,853

📈 Phân bố theo Sentiment:
Sentiment
positive    15937
negative     3116
neutral      1788

ÁP DỤNG CÂN BẰNG DỮ LIỆU
  positive: 15,937 → 3,000 (giảm 12,937)
  negative

In [32]:
# Xem một số mẫu dữ liệu
print("📋 Một số mẫu dữ liệu đầu tiên:")
merged_df.head(10)

📋 Một số mẫu dữ liệu đầu tiên:


,reviewText,AspectTerm,Sentiment
0,"While the tablet is nice, and well constructed...",google play,neutral
1,Highly recommended for those looking for that ...,food recommendation,positive
2,"ZICO Pure Premium Coconut Water , Natural , 11...",food general,positive
3,This is the only time I have strongly disliked...,food quality,negative
4,"Located on Via Della Scala , it was the perfec...",location general,positive
5,"It works in some, not all USB ports because it...",installation,neutral
6,"I LOVE PB , but try not to eat too much becaus...",food general,neutral
7,Excellent Booked this hotel a couple of months...,hotel general,positive
8,McCormick spices aren ' t exactly world - reno...,food general,neutral
9,"First thing , the room was really small and sq...",rooms design_features,negative


## Tổng kết

File đã được tạo với cấu trúc 3 cột:
- `reviewText`: Nội dung review
- `AspectTerm`: Aspect term được trích xuất
- `Sentiment`: Nhãn sentiment (positive/negative/neutral)